[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/real-itu/modern-ai-course/blob/master/lecture-03/lab.ipynb)

# Lab 3 - Monte Carlo Tree Search

In this exercise we will use the same game as in the previous exercise, namely, Connect 4.
([Connect 4](https://en.wikipedia.org/wiki/Connect_Four)). You should implement the MCTS algorithm to play the game.

As before, the game is implemented below. It will play a game where both players take random (legal) actions. The MAX player is represented with a X and the MIN player with an O. The MAX player starts. Execute the code.

In [86]:
import random
from copy import deepcopy
from typing import Sequence

NONE = '.'
MAX = 'X'
MIN = 'O'
COLS = 7
ROWS = 6
N_WIN = 4


class ArrayState:
    def __init__(self, board, heights, n_moves):
        self.board = board
        self.heights = heights
        self.n_moves = n_moves

    @staticmethod
    def init():
        board = [[NONE] * ROWS for _ in range(COLS)]
        return ArrayState(board, [0] * COLS, 0)


def result(state: ArrayState, action: int) -> ArrayState:
    """Insert in the given column."""
    assert 0 <= action < COLS, "action must be a column number"

    if state.heights[action] >= ROWS:
        raise Exception('Column is full')

    player = MAX if state.n_moves % 2 == 0 else MIN

    board = deepcopy(state.board)
    board[action][ROWS - state.heights[action] - 1] = player

    heights = deepcopy(state.heights)
    heights[action] += 1

    return ArrayState(board, heights, state.n_moves + 1)


def actions(state: ArrayState) -> Sequence[int]:
    return [i for i in range(COLS) if state.heights[i] < ROWS]


def branch_states(state: ArrayState) -> Sequence[ArrayState]:
    """get all reachable states from the current state:
        useful for MCTS
    """
    return [result(state, a) for a in actions(state)]
    

def utility(state: ArrayState) -> float:
    """Get the winner on the current board."""

    board = state.board

    def diagonalsPos():
        """Get positive diagonals, going from bottom-left to top-right."""
        for di in ([(j, i - j) for j in range(COLS)] for i in range(COLS + ROWS - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < COLS and j < ROWS]

    def diagonalsNeg():
        """Get negative diagonals, going from top-left to bottom-right."""
        for di in ([(j, i - COLS + j + 1) for j in range(COLS)] for i in range(COLS + ROWS - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < COLS and j < ROWS]

    lines = board + \
            list(zip(*board)) + \
            list(diagonalsNeg()) + \
            list(diagonalsPos())

    max_win = MAX * N_WIN
    min_win = MIN * N_WIN
    for line in lines:
        str_line = "".join(line)
        if max_win in str_line:
            return 1
        elif min_win in str_line:
            return -1
    return 0


def terminal_test(state: ArrayState) -> bool:
    return state.n_moves >= COLS * ROWS or utility(state) != 0


def printBoard(state: ArrayState):
    board = state.board
    """Print the board."""
    print('  '.join(map(str, range(COLS))))
    for y in range(ROWS):
        print('  '.join(str(board[x][y]) for x in range(COLS)))
    print()

def playerTurn(state: ArrayState):
    return MAX if state.n_moves % 2 == 0 else MIN

s = ArrayState.init()
playRandom = False
while not terminal_test(s):
    if playerTurn(s) == MIN:
        a = -1
        if playRandom:
            a = random.choice(actions(s))
        else:
            validInput = False
            while(not validInput):
                a = int(input("Please Input a column!"))
                validInput = a in actions(s)
                if(not validInput):
                    print("Not a valid column. Try again!")
    else:
        mcts = MCTS()
        a = mcts.choose(s, 0.01)
    s = result(s, a)
    printBoard(s)
    print(utility(s))


0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
X  .  .  .  .  .  .

0
0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
X  .  .  O  .  .  .

0
0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
X  .  .  O  X  .  .

0
0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  O  .  .  .
X  .  .  O  X  .  .

0
0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  O  X  .  .
X  .  .  O  X  .  .

0
0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  O  .  .  .
.  .  .  O  X  .  .
X  .  .  O  X  .  .

0
0  1  2  3  4  5  6
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  .  .  .  .
.  .  .  O  X  .  .
.  .  .  O  X  .  .
X  .  .  O  X  .  .

0

The last number 0, -1 or 1 is the utility or score of the game. 0 means it was a draw, 1 means MAX player won and -1 means MIN player won.

### Exercise 1 (Transfer code from the previous exercise)

Modify the code so that you can play manually as the MIN player against the random AI.

In [ ]:
### Code here!

### Exercise 2
Implement the standard MCTS algorithm.

In [ ]:
class Node:
    def __init__(self, parent, action: int, state : ArrayState):
        self.parent = parent
        self.state = state
        self.action = action
        self.children = []
        self.N = 0
        self.Q = 0

    def addChild(self, child):
        self.children.append((child))

    def getUsedActions(self) -> Sequence[int]:
        return list(map(lambda c : c.action, self.children))

    def fullyExpanded(self) -> bool:
        isFully = (len(self.children) == len(actions(self.state)))
        return isFully

In [ ]:
from abc import ABC, abstractmethod
from collections import defaultdict
import math
from datetime import datetime
import random

class MCTS:
    "Monte Carlo tree searcher."

    def __init__(self, exploration_weight=1):
        pass

    def do_rollout(self, state : ArrayState) -> float:
        "Train for one iteration."
        while not terminal_test(state):
            acts = actions(state)
            a = random.choice(acts)
            state = result(state, a)
        return utility(state)

    def _expand(self, node : Node) -> Node:
        "Expand the `state` with all states reachable from it"
        acts = actions(node.state)
        expanded = node.getUsedActions()
        unexpanded = list(set(acts) - set(expanded))
        action = unexpanded.pop()
        child = Node(node, action, result(node.state, action))
        node.addChild(child)
        return child

    def _uct_select(self, node : Node):
        "Select a child of state, balancing exploration & exploitation"
        q = -float('inf')
        bestChild = None
        for c in node.children:
            newQ = (c.Q / c.N) + (math.sqrt((2 * math.log(node.N)) / c.N))
            if newQ > q:
                q = newQ
                bestChild = c            
        return bestChild

    def _select(self, node : Node) -> Node:
        "Find an unexplored descendent of the `state`"
        while not terminal_test(node.state):
            if not node.fullyExpanded():
                return self._expand(node)
            else:
                node = self._uct_select(node)
        return node

    def _simulate(self, state : ArrayState) -> float:
        "Returns the reward for a random simulation (to completion) of the `state`"
        return self.do_rollout(state)
    
    def _backpropagate(self, node : Node, reward):
        "Send the reward back up to the ancestors of the leaf"
        while not node == None:
            node.N += 1
            node.Q += reward
            node = node.parent

    def choose(self, state : ArrayState, maxTime: float) -> int:
        "Choose  a move in the game and execute it"
        start = datetime.now()
        root = Node(None, None, state)
        while (datetime.now() - start).seconds < maxTime:
            v = self._select(root)
            d = self._simulate(v.state)
            self._backpropagate(v, d)
        bestNode = self._uct_select(root)
        return bestNode.action



### Exercise 3
Implement the loop where you play against your MCTS agent. Either train the agent at each step while you play against it, or pretrain it with more rollouts and play agaist it after training.

In [ ]:
def train_model(state : ArrayState, num_rollouts : int):
    pass

In [ ]:
## Play against the MCTS agent
pass


### Exercise 4

Add move ordering. The middle columns are often "better" since there's more winning positions that contain them. Increase the probability to choose middle columns when randomly executing rollouts: [3,2,4,1,5,0,6]. See if your connect4 AI can beat you.


### Exercise 5 - Optional

Pit your MCTS agent against the one from the previous exercise.
* Which one wins more often?
* Which one takes more time to run per step once it is at a point that it can beat you?